Install required packages

In [ ]:
%pip install -q -r requirements.txt

Import packages and show versions

In [ ]:
import sys
import numpy as np

print("Python ", sys.version)
print("Numpy ", np.__version__)

Set up transformer

"SentenceTransformer", is the SentenceTransformer class is part of the SentenceTransformers library, which is designed to simplify the process of generating semantically meaningful embeddings for sentences, paragraphs, or larger texts.


_all-MiniLM-L6-v2_ is a MiniLM model fine tuned on a large dataset of over 1 billion training pairs

In [11]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("all-MiniLM-L6-v2")

In [ ]:
sentences = [
    "We may brave human laws, but we cannot resist natural ones.",
    "Imagine a society in which there were neither rich nor poor",
    "If Providence has created the stars and the planets, man has called the cannonball into existence",
    "The Yankees, the first mechanicians in the world, are engineers - just as the Italians are musicians and the Germans metaphysicians - by right of birth. Nothing is more natural, therefore, than to perceive them applying their audacious ingenuity to the science of gunnery.",
]
embeddings = model.encode(sentences)
print(embeddings.shape)

print(embeddings.tolist())

(4, 384)
1536


Next step,

build query/vector search.

mongodb vectorSearch, pipeline, match, group, sort and limit